In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras as ks

from tensorflow.python.data.experimental import AUTOTUNE


2022-05-17 16:45:04.121865: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
# specify data directories
data_path = './mtg.tfrecords'

input_shape = [1040, 745, 3]

In [3]:
# load tfrecords dataset
image_feature_description = {
    "image": tf.io.FixedLenFeature([], tf.string), 
    "class": tf.io.FixedLenFeature([], tf.int64), 
    }

def _parse_data(unparsed_example):
    return tf.io.parse_single_example(unparsed_example, image_feature_description)

def _bytestring_to_pixels(parsed_example):
    byte_string = parsed_example['image']
    image = tf.io.decode_image(byte_string)
    image = tf.reshape(image, input_shape)
    return image, parsed_example["class"]

def load_and_extract_images(filepath):
    dataset = tf.data.TFRecordDataset(filepath)
    dataset = dataset.map(_parse_data, num_parallel_calls=AUTOTUNE)
    dataset = dataset.map(_bytestring_to_pixels, num_parallel_calls=AUTOTUNE) # .cache()
    return dataset

def tripletloss(model_anchor, model_positive, model_negative, margin=2):
    distance1 = tf.sqrt(tf.reduce_mean(tf.pow(model_anchor - model_positive, 2), 1, keepdims=True))
    distance2 = tf.sqrt(tf.reduce_mean(tf.pow(model_anchor - model_negative, 2), 1, keepdims=True))
    return tf.reduce_mean(tf.maximum(distance1 - distance2 + margin, 0))

In [12]:
dataset = load_and_extract_images(data_path)

In [14]:
def triplet_input_fn(dataset):
    double_set = tf.data.Dataset(dataset.batch(2))
    return double_set

triplet_set = triplet_input_fn(dataset)

TypeError: Can't instantiate abstract class DatasetV2 with abstract methods _inputs, element_spec

In [ ]:
from matplotlib.pyplot import imshow
test_set = train_dataset.take(1)
for image, label in test_set:
    print(image.shape)
    imshow(image)

In [ ]:
convnet = ks.Sequential()
convnet.add(ks.layers.Input(input_shape))
convnet.add(ks.layers.Conv2D(32, [7,7]))
convnet.add(ks.layers.MaxPool2D())
convnet.add(ks.layers.Conv2D(64, [5,5]))
convnet.add(ks.layers.MaxPool2D())
convnet.add(ks.layers.Conv2D(128, [3,3]))
convnet.add(ks.layers.MaxPool2D())
convnet.add(ks.layers.Flatten())
convnet.add(ks.layers.Dense(256, activation=None))
convnet.add(ks.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)))

In [ ]:
convnet.summary()

In [ ]:
convnet.compile(optimizer='Adam', loss=tripletloss, metrics=['accuracy'])

In [ ]:
for epoch in range(10):
    convnet.fit(dataset, batch_size=8)